In [ ]:
from clifford.sta import * 
from pylab import * 

if True:
    
    thres       = 1e-10
    is_close    = lambda x,y:abs(x-y)<thres
    is_close_ish= lambda x,y:abs(x-y)<thres*1e4
    split       = lambda a,B: (a|B)/(a^B)
    split_apart = lambda a,B: ((a|B)/B, (a^B)/B)
    nudge       = lambda a: a+1e-5 if a == 0 else a
    
    # constants not defined by model
    m    = rand()                   
    q    = rand()
    hbar = rand()
    
    # basis elements in standard frame
    S = d12
    i = d123 
    I = d0123
    dk=k = (i*S)# +.01*d1
    k=k/abs(k)
    
    C = I*S
    pis = pi*S  # == \pi_S
    
    
    ## note: since we dont have a nice way to write the dot notation 
    # we have set the differential (rdot) as rd ,  
    #rd = d0 + .2*d1 + .84*d2 + .3*d3 # known differential
    rd = D.randomV()            # random differential.
    rd = rd(i) + abs(rd(i))*d0  # put it on the light cone
    
    
    # differential components
    rd0,rdk,rds = rd(d0),rd(k),rd(S)
    rdsnot = rd-rds
    rd0not = rd-rd0
    
    # velocities
    Vs = rds/rd0
    Vk = rdk/rd0
    V  = Vs + Vk
    T  = Vk/Vs
    c  = abs(V)
    
    # linear momentum 
    ls,lk  = m*Vs,m*Vk  
    l = ls+lk

    ## IMPORTANT,  most tests hinge on this
    # this is compton wavelength *1/gamma^2
    rs = hbar/m*abs(Vs) * (rds|S / abs(rds|S))  # position vector in S 
    #        mag                direction rs|rds==0
    
    # frequencies, angular momenta
    theta = (rds/rs)
    W = (rds/rs)/rd0
    K = (rds/rdk)/rs
    P = W + K
    p,ps,pk = m*V,m*Vs, m*Vk
    L = m*rs^Vs
    E = m*V**2

    pis = pi*S
    # wavelengths
    lam   = -2*pi*S*V/W
    lam_s = -2*pi*S*Vs/W
    lam_k = -2*pi*S*Vk/W
    
    # currents
    J  = q*V/lam
    Js = q*Vs/lam_s
    Jk = q*Vk/lam_k
    
    # lorentz factor and helix space pitch 
    Gamma = V*Vs.inv()         # spinor form 
    gamma = abs(Gamma)   # lorentz factor
    phi = arctan(abs(Vk)/abs(Vs))
    Le = gamma**2 *L
    
    # electrodynamics
    M = Js*pis*rs**2
    B = m/q*W
    mu_ring = 4*pis*m*rs/q**2
    Phi= B*pis*rs**2

if True:
    ################ TESTS ############################
    
    # Velocity and Differentials
    assert(rd == rd0+rdk+rds == rds+rdsnot)
    assert(-theta/(rd0*rdk)*(rd0+rdk) == P)      # P-r relation
    assert(  -rs*P == (rds*rdsnot)/(rd0*rdk))    # P-r relation
    assert(  Vs+Vk == rd0not/rd0)
    
    # Angular Momentum
    assert(L == ps^rs == -m*rs**2*W == -m*Vs**2/W)
    assert(isclose(abs(rs),abs(L/(m*Vs)))) # 
    assert(  rs**2 == - L/W/m)             # radius, as f(L,W,m)
    assert(T==Vk/Vs == rdk/rds== 1/(K*rs))
    assert(isclose(abs(T),tan(phi)))
    assert(Gamma==1+T)
     
    # Wavelengths
    assert(  lam == 2*pi*S*(rs+1/K)==-2*pis*V/W)    # lamda = krds one  
    assert(lam_s == 2*pi*S*rs== 2*pis*L/ps==-2*pis*Vs/W) # deBroglie
    assert(lam_k == 2*pi*S*(1/K)==-2*pis*Vk/W)
    assert(isclose(abs(lam)**2 ,abs(lam_s)**2 +abs(lam_k)**2))
    assert(lam==lam_s+lam_k)
    
    
    # Currents 
    assert(Js == -Jk ==-q*W/(2*pis))
    assert(isclose(abs(J),abs(Js)))
    assert(isclose(abs(J),abs(Jk)))
    assert(J==-q*W/(2*pis)*((1+T)/(1-T)))
    assert(J==-q*W/(2*pis)*V/(-k*V*k))
    assert(tan(phi)*(Vk/Vs)(2).normal() ==Vk/Vs)
    
    
    # Lorentz factor 
    assert(V**2 == Vs**2 +Vk**2)
    assert(Gamma==V*Vs.inv() == 1/(1-Vk*V.inv()))
    assert(is_close(gamma,1/sqrt(1-abs(Vk**2/V**2))))
    assert(is_close(gamma,1/cos(phi)))
       
    # Energy
    assert(E == m*V**2 == -L*W + m*Vk**2) # energy momentum
    assert(E ==-Le*W==-gamma**2*L*W)  
    assert(m*Vs**2 == -L*W )    


    # dirac equation 
    x    = D.randomV()
    psi  = e**(P*x)  # not sure about this. 
    dpsi = P*psi     # TODO: this shoudl not be a def, but derived from calc
    A    = E/(q*Vk) 
    
    assert(Le == abs(Le)*d012) # interpretation of Le as hbar 
    assert( dpsi== P*psi == (W+K)*psi == W*(1-1/Vk)*psi == -E/Le*(1-1/Vk)*psi) 
    assert(Le*dpsi -E/Vk*psi ==-E*psi ) 
    assert(1/c*Le*dpsi - q/c *E/(q*Vk)*psi == -m*c*psi)
    assert(A==E/(q*Vk) == m*V/q *(1+Vs/Vk))
    assert(d12*hbar* 1/c *dpsi - d0*q/c*A*psi == -d0*m*c*psi)
    #assert(1/c*abs(Le)*d12*i*dpsi*i +q/c*E/(q*abs(Vk)*d3)*i*psi*i == m*c*psi*d0) #fix 
   # assert(1/c*hbar*d12*i*dpsi*i +q/c*E/(q*abs(Vk)*d3)*i*psi*i == m*c*psi*d0) # fix 
    assert(-(psi*P)*d12*abs(Le) +E/(abs(Vk)*dk)* (dk*psi/dk) == E*psi*d0)  # solution if dpsi = psi*P 
    #not used 
    assert(1/c*Le*dpsi == -1/c*abs(Le)*d12*i*dpsi*i*d0 == d0/c*abs(Le)*d12*dpsi)
    #assert(q/c *E/(q*Vk)*psi ==  q/c *E/(q*abs(Vk)*d3)*i*psi*i*d0== -d0*q/c *E/(q*abs(Vk)*d3)*psi )  #second part
    
    
    
    ## diracs original formula
    dpsidt = W*psi
    assert(dpsidt == -E/Le*psi )
    assert(-Le*dpsidt == (-1/gamma**2*Le*W + Vk*pk)*psi)
   # assert(Vk == -abs(V)*sqrt(1-1/gamma**2)*d3*d0) # dirac original
    #assert(-d12*hbar*dpsidt== (d0/gamma**2*m*c**2 + (c*sqrt(1-1/gamma**2))*d3*pk)*psi)
if True:
 
    # Unfinished
    ## symmetries
    assert(-d0*P*d0 == -W + K) # T 
    assert(-i*P*i   == +W - K) # P?
    assert(-I*P*I   == -W - K) # C?
    assert(split(P*I,d0)== split(rdsnot,d0))

    ## null basis trivectords
    d0pkS = .5*(d0+d3)^S 
    d0mkS = .5*(d0-d3)^S
    Pp = (abs(W)+abs(K))*d0pkS  # probably a slicker way to do this
    Pm = (abs(W)-abs(K))*d0mkS
    #assert(P == Pm+Pp)
    

 